# Python Programming for GBN Implementation
## Building Packing with bytearray

In [1]:
a = bytes(b'abcdedg')     # immutable
b = bytearray(b'abcdefg') # mutable
b[1] = 66
print(b)
print(b[1])               # int
print(b[1:])

bytearray(b'aBcdefg')
66
bytearray(b'Bcdefg')


In [2]:
c = bytearray([65, 66, 67, 68, 69, 70, 71])
print(c)

bytearray(b'ABCDEFG')


In [3]:
ACK = 1
seq = 14
message = 'Hello, world'
data = message.encode('utf-8')
print(type(data))
packet = bytearray([ACK, seq, 0, 0])    # header
packet.extend(data)  # just like list
print(packet)
print(packet[1])

<class 'bytes'>
bytearray(b'\x01\x0e\x00\x00Hello, world')
14


## Internet Checksum
Packet을 16-bit integer의 열로 봐서 checksum을 계산한다. 보통 header에 checksum field로 16 bit가 할당되어 있다.

이 Checksum field를 0으로 clear해 놓고 checksum을 계산한 후 결과를 checksum field에 삽입한다. Checksum이 들어 있는 packet에 대해 다시 checksum을 계산하면 결과는 0. 

원래 packet의 checksum을 ```sum```이라 하자. Checksum field에 ```sum```을 넣으면 
이 만큼 더해진다. 1's complement 연산에서 덧셈은 뺄셈과 같다. 그러므로
```Python
sum + sum == sum - sum == 0
``` 

In [5]:
from packet import ichecksum

checksum = ichecksum(packet)
packet[2:4] = bytearray([checksum >> 8, checksum & 0xFF])
print(packet)
print(ichecksum(packet))

bytearray(b'\x01\x0e\xde\xa5Hello, world')
0


## Packet Sequence Number
Packet sequence number는 header 내에 하나의 field로 표현해야 한다. 예를 들어, 1 byte로 표현하면 0 ~ 255의 수로만 표현할 수 있다. 255 다음 sequence는 0으로 돌아 와야 한다. 다음과 같은 연산이 가능한 ```Seq```라는 class, 다시 말해, 새로운 data type을 정의해 보자.

```Python
>>> Seq(255) + Seq(1)
Seq(0)
>>> Seq(255) + 8
Seq(7)
>>> Seq(255) < Seq(7)
True
``` 

In [6]:
from packet import Seq

base = Seq(253)        # 'Seq' class instance, i.e. object
nextseqnum = Seq(2)    # anothor class instance
print(base.seq)        # attribute 'seq' of 'base' object
print(base)
print(type(base))      # tell the class belongs to

253
Seq(253)
<class 'packet.Seq'>


#### ```repr, str, print``` functions
repr(): calls __repr__ method

str(): calls __str__ method
- calls __repr__ if __str__ is not defined

print(): calls str()

### Magic methods
미리 정의된 이름이으로 ```__``` 이 앞과 뒤에 붙는다. ```+, -, *, /, ==, <=, ...``` 등의 arithmetic, comparison operator를 사용하여 expression을 기술하면, 해당되는 magic method를 call하게 된다.

#### ```int``` function

In [8]:
print(Seq.__int__(base))  # function call stype
print(base.__int__())     # invoke method of base object
print(int(base))          # int function calls base.__int__()

253
253
253


#### Addition

```Python
seq + n     # forward n steps
seq1 + seq2 # TypeError
```

In [13]:
print(base)
nextseqnum = base + 5
print(nextseqnum)
nextseqnum += 1
print(nextseqnum)

Seq(253)
Seq(2)
Seq(3)


#### Comparison

In [15]:
N = 8
if nextseqnum < base + N:
    print('send it')
else:
    print('refuse data')

acknum = Seq(254)
if base <= acknum < base + N:
    print('legal ACK')

send it
legal ACK


### Seq range

In [16]:
from packet import srange

for s in srange(base, nextseqnum):
    print(s)

Seq(253)
Seq(254)
Seq(255)
Seq(0)
Seq(1)
Seq(2)


## Handling Packets
### Making and sending packets - GBN sending side

In [17]:
def make_pkt(seq, packet_type, data=b''):
    if not isinstance(data, (bytes, bytearray)):
        raise TypeError('data is not bytes or bytearray type')
    packet = bytearray([packet_type, int(seq), 0, 0])  # header with zero checksum field
    packet.extend(data)
    checksum = ichecksum(packet)
    packet[2:4] = bytearray([checksum >> 8, checksum & 0xFF])  # overwrite checksum
    return packet

nextseqnum = base = Seq(254)   # empty buffer
sndpkt = []

# Send 4 packets
DATA = 1    # DATA packet type
data = b'Hello, world!!'
for i in range(4):
    packet = make_pkt(nextseqnum, DATA, data)
    sndpkt.append(packet)          # send buffer for retransmission
    if base == nextseqnum:    # if send buffer was empty
        print('start_timer()')
    print('udt_send:', packet)
    nextseqnum += 1

print(base, nextseqnum)
print('sndpkt:', sndpkt)

start_timer()
udt_send: bytearray(b'\x01\xfe\xbc\x94Hello, world!!')
udt_send: bytearray(b'\x01\xff\xbc\x93Hello, world!!')
udt_send: bytearray(b'\x01\x00\xbd\x92Hello, world!!')
udt_send: bytearray(b'\x01\x01\xbd\x91Hello, world!!')
Seq(254) Seq(2)
sndpkt: [bytearray(b'\x01\xfe\xbc\x94Hello, world!!'), bytearray(b'\x01\xff\xbc\x93Hello, world!!'), bytearray(b'\x01\x00\xbd\x92Hello, world!!'), bytearray(b'\x01\x01\xbd\x91Hello, world!!')]


### ACK handling
Packet n을 받으면, ACK n을 feedback 하기 보다 ACK n+1 으로 응답하는 것이 보다 논리적이고 coding이 쉽다. 이 의미는 packet ..., n-1, n 은 순서대로 잘 받았고, n+1 번 packet을 받을 차례임을 sender에게 알리는 것이다. 따라서, 송신측은 항상 ACK 번호부터 retransmit하면 된다. 실제 대부분의 프로토콜들은 이렇게 정의되어 있다.
#### GBN receiving side

In [19]:
expectedseqnum = Seq(254)

expectedseqnum += 1
sndpkt = make_pkt(expectedseqnum, ACK)
print('udt_send:', sndpkt)

udt_send: bytearray(b'\x01\xff\xfe\x00')


#### GBN sending side

In [22]:
acknum = Seq(255)    # 255번 packet 받을 차례. 254번까지는 잘 받았음.
for i in srange(base, acknum): # The packets < acknum already correctly received.
    del sndpkt[0]              # So, delete packets such that self.base <= seq < acknum.
base = acknum
if base == nextseqnum:
    print('stop_timer()')
# else:
#     self.start_timer()